In [14]:
from torch import nn
import torch
import sys
from open_clip.loss import DistillClipLoss
sys.path.append('..')
from mae.models_mae import MaskedAutoencoderViT, partial
sys.path.append('../..')
from DUL294P.encoders.openclip_encoder import *


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters()) / 1e6

In [24]:
count_parameters(clipencoder.model.visual)

86.19264

In [3]:
clipencoder = OpenCLIPNetworkConfig(device='cpu').setup()

In [4]:
model = MaskedAutoencoderViT(
        patch_size=16, embed_dim=768, depth=12, num_heads=12,
        decoder_embed_dim=512, decoder_depth=8, decoder_num_heads=16,
        mlp_ratio=4, out_chans=768, norm_layer=partial(nn.LayerNorm, eps=1e-6), return_pooled=True)
sd=torch.load('../models/mae_pretrain_vit_base_full.pth')['model']
msg = model.load_state_dict(sd, strict=False)
print(msg)

img = torch.randn(1, 3, 224, 224)
target = model.patchify(img)
latent, mask, ids_restore = model.forward_encoder(img, .75)
pred, pooled = model.forward_decoder(latent, ids_restore)
pred.shape, pooled.shape

_IncompatibleKeys(missing_keys=['pool_proj.weight', 'pool_proj.bias'], unexpected_keys=[])


(torch.Size([1, 196, 768]), torch.Size([1, 512]))

In [ ]:
import time
img = torch.randn(8, 3, 224, 224)
times = []
model.float()
model.to("cpu")
with torch.no_grad():
  for _ in range(10):
    img.normal_()
    stime = time.time()
    # target = model.patchify(img)
    latent, mask, ids_restore = model.forward_encoder(img.to("cpu"), 0.75)
    # pred, pooled = model.forward_decoder(latent, ids_restore)
    etime = time.time()
    times.append(etime-stime)
print(sum(times)/len(times))

In [ ]:
0.015513920783996582
0.008359954357147217

In [92]:
import time
img = torch.randn(8, 3, 224, 224)
clipencoder.float().to("cuda:1")
times = []
with torch.no_grad():
  for _ in range(100):
    img.normal_()
    stime = time.time()
    clipencoder.model.encode_image(img.to("cuda:1"))
    etime = time.time()
    times.append(etime-stime)
print(sum(times)/len(times))

0.013010034561157227


In [60]:
count_parameters(model), count_parameters(clipencoder.model.visual)

(112.301568, 86.19264)

In [5]:
txt = clipencoder.model.encode_text(clipencoder.tokenizer(['a photo of a cat',]), normalize=True)

In [6]:
clipencoder.process

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
)

In [7]:
img_prepro = clipencoder.process(img).half()
pooled_clp, pred_clp = clipencoder.model.encode_image(img_prepro)
pred_clp.shape, pooled_clp.shape

(torch.Size([1, 196, 768]), torch.Size([1, 512]))

In [17]:
(DistillClipLoss().get_logits(
  pooled.float(), txt.float(), clipencoder.model.logit_scale, 
  pooled_clp.float(), txt.float(), clipencoder.model.logit_scale))

(tensor([[-0.0165]], grad_fn=<MmBackward0>),
 tensor([[-0.0165]], grad_fn=<MmBackward0>))